### imports

In [ ]:
import sys
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.patches import Circle


import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
# important to run this "magic" for things to display nicely!

In [ ]:
%matplotlib notebook 

# What are widgets?

"Widgets are eventful python objects that have a representation in the browser, often as a control like a slider, textbox, etc."

They're another level of interactivity.

## Introduction to a few types of widgets

There are *tons* of widgets out there! The landscape can get a bit confusing, so let's just walk through a few of my representative favoritess

In [ ]:
widgets.IntSlider()

In [ ]:
w = widgets.IntSlider()
display(w)

In [ ]:
# these two are linked together! different *references* to same underlying object.
# that object has its own attribute — slider.value — that will change when we slide.
# so each view view appropriately change, as well!

display(w)


In [ ]:
widgets.Text(value='Hellooooo World!', disabled=True)


In [ ]:
# possible to link two widgets together explicitly

a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)

mylink = widgets.dlink((a, 'value'), (b, 'value'))

In [ ]:
# problem: don't want to keep pinging server? link in browser!

a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)

mylink = widgets.jslink((a, 'value'), (b, 'value'))

### Exercise: link an integer slider to a float slider! Does it break?

### Exercise: how is this different from viewing the same widget twice?

There are a bunch of keyword arguments that we can exploit, as well!

In [ ]:
widgets.FloatSlider(
    value=7,
    min=2,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

### Exercise: make a vertical float slider that ranges from 0 to 100, steps of 5, and starts at 2!

In [ ]:
widgets.IntProgress(
    value=7,
    min=0,
    max=10,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)

In [ ]:
widgets.ToggleButton(
    value=False,
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [ ]:
widgets.Dropdown(
    options=['1', '2', '3'],
    value='2',
    description='Number:',
    disabled=False,
)

In [ ]:
widgets.ColorPicker(
    concise=False,
    description='Pick a color',
    value='blue',
    disabled=False
)

## Event handlers

This is how we can register "callback functions."

In [ ]:
btn = widgets.Button(description='Test')
other_btn = widgets.Button(description='Other')
display(btn)
display(other_btn)

def my_event_handler(btn_object):
    print('You pressed {}'.format(btn_object.description))
    
btn.on_click(my_event_handler)
other_btn.on_click(my_event_handler)

# Plotting

In [ ]:
slider1 = widgets.IntSlider(
                            value=0,
                            min=0,
                            max=30,
                            step=2,
                            description=r'\(N_{\rm points}\)')

def make_plot(numpoints=0):
    
    plt.cla()
    # Plot and save an image of the double pendulum configuration for time
    # point i.
    # The pendulum rods.
    
    ax.plot(np.random.random(size=numpoints),
           color='gray',
           linestyle='--')
    
    plt.xticks([])
    plt.yticks([])


out = widgets.interactive_output(make_plot, {'numpoints':slider1,
                                     })


fig = plt.figure(figsize=(8,6), 
                 dpi=72)

plt.axis('off')


ax = fig.add_subplot(111)



# we can add vertical and horizontal boxes for styling!
box = widgets.VBox([slider1])


ui = widgets.HBox([box])

display(ui, out)
make_plot()

## Discussion: What are some other cases in which this could come in handy? 

# More advanced Plotting

source code adapted from https://scipython.com/blog/the-double-pendulum/

In [ ]:
# Pendulum rod lengths (m), bob masses (kg).
L1, L2 = 1, 1
m1, m2 = 1, 1
# The gravitational acceleration (m.s-2).
g = 9.81

def deriv(y, t, L1, L2, m1, m2):
    """Return the first derivatives of y = theta1, z1, theta2, z2."""
    theta1, z1, theta2, z2 = y

    c, s = np.cos(theta1-theta2), np.sin(theta1-theta2)

    theta1dot = z1
    z1dot = (m2*g*np.sin(theta2)*c - m2*s*(L1*z1**2*c + L2*z2**2) -
             (m1+m2)*g*np.sin(theta1)) / L1 / (m1 + m2*s**2)
    theta2dot = z2
    z2dot = ((m1+m2)*(L1*z1**2*s - g*np.sin(theta2) + g*np.sin(theta1)*c) + 
             m2*L2*z2**2*s*c) / L2 / (m1 + m2*s**2)
    return theta1dot, z1dot, theta2dot, z2dot

def calc_E(y):
    """Return the total energy of the system."""

    th1, th1d, th2, th2d = y.T
    V = -(m1+m2)*L1*g*np.cos(th1) - m2*L2*g*np.cos(th2)
    T = 0.5*m1*(L1*th1d)**2 + 0.5*m2*((L1*th1d)**2 + (L2*th2d)**2 +
            2*L1*L2*th1d*th2d*np.cos(th1-th2))
    return T + V

# Maximum time, time point spacings and the time grid (all in s).
tmax, dt = 30, 0.01
t = np.arange(0, tmax+dt, dt)
# Initial conditions: theta1, dtheta1/dt, theta2, dtheta2/dt.
y0 = np.array([3*np.pi/7, 0, 3*np.pi/4, 0])

# Do the numerical integration of the equations of motion
y = odeint(deriv, y0, t, args=(L1, L2, m1, m2))

# Check that the calculation conserves total energy to within some tolerance.
EDRIFT = 0.05
# Total energy from the initial conditions
E = calc_E(y0)
if np.max(np.sum(np.abs(calc_E(y) - E))) > EDRIFT:
    sys.exit('Maximum energy drift of {} exceeded.'.format(EDRIFT))

# Unpack z and theta as a function of time
theta1, theta2 = y[:,0], y[:,2]

# Convert to Cartesian coordinates of the two bob positions.
x1 = L1 * np.sin(theta1)
y1 = -L1 * np.cos(theta1)
x2 = x1 + L2 * np.sin(theta2)
y2 = y1 - L2 * np.cos(theta2)



fps = 10
di = int(1/fps/dt)

# Plotted bob circle radius
r = 0.05
# Plot a trail of the m2 bob's position for the last trail_secs seconds.
trail_secs = 1
# This corresponds to max_trail time points.
max_trail = int(trail_secs / dt)




color1 = widgets.ColorPicker(
    concise=False,
    description='Top weight color',
    value='red',
    disabled=False
)

color2 = widgets.ColorPicker(
    concise=False,
    description='Bottom weight color',
    value='red',
    disabled=False
)


edgecolor1 = widgets.ColorPicker(
    concise=False,
    description='Top edge color',
    value='red',
    disabled=False
)

edgecolor2 = widgets.ColorPicker(
    concise=False,
    description='Bottom edge color',
    value='red',
    disabled=False
)


timesteps =widgets.IntSlider(
                                            value=0,
                                            min=0,
                                            max=t.size,
                                            step=di,
                                            description=r'time',)

def make_plot(i=0, 
              color1='red',
              edgecolor1='red',
              color2='red',
              edgecolor2='red'):
    
    plt.cla()
    # Plot and save an image of the double pendulum configuration for time
    # point i.
    # The pendulum rods.
    ax.plot([0, x1[i], x2[i]], [0, y1[i], y2[i]], lw=2, c='k')
    # Circles representing the anchor point of rod 1, and bobs 1 and 2.
    c0 = Circle((0, 0), r/2, fc='k', zorder=10)
    c1 = Circle((x1[i], y1[i]), r, fc=color1, ec=edgecolor1, zorder=10)
    c2 = Circle((x2[i], y2[i]), r, fc=color2, ec=edgecolor2, zorder=10)
    ax.add_patch(c0)
    ax.add_patch(c1)
    ax.add_patch(c2)

    # The trail will be divided into ns segments and plotted as a fading line.
    ns = 20
    s = max_trail // ns

    for j in range(ns):
        imin = i - (ns-j)*s
        if imin < 0:
            continue
        imax = imin + s + 1
        # The fading looks better if we square the fractional length along the
        # trail.
        alpha = (j/ns)**2
        ax.plot(x2[imin:imax], y2[imin:imax], c='r', solid_capstyle='butt',
                lw=2, alpha=alpha)

    # Centre the image on the fixed anchor point, and ensure the axes are equal
    ax.set_xlim(-L1-L2-r, L1+L2+r)
    ax.set_ylim(-L1-L2-r, L1+L2+r)
    ax.set_aspect('equal', adjustable='box')


out = widgets.interactive_output(make_plot, {'i':timesteps,
                                             'color1': color1,
                                     'color2': color2,
                                             'edgecolor1': edgecolor1,
                                             'edgecolor2': edgecolor2,
                                     })

# Make an image every di time points, corresponding to a frame rate of fps
# frames per second.
# Frame rate, s-1

fig = plt.figure(figsize=(8.3333, 6.25), dpi=72)
plt.axis('off')
ax = fig.add_subplot(111)

left_box = widgets.VBox([color1, color2])
middle_box = widgets.VBox([edgecolor1, edgecolor2])
right_box = widgets.VBox([timesteps])


ui = widgets.HBox([left_box,middle_box, right_box])

display(ui, out)
make_plot()

# for i in range(0, t.size, di):
#     print(i // di, '/', t.size // di)
#     make_plot(i)

# Filtering your data
source: https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916

In [ ]:
import pandas as pd
import numpy as np
url = "https://data.london.gov.uk/download/number-international-visitors-london/b1e0f953-4c8a-4b45-95f5-e0d143d5641e/international-visitors-london-raw.csv"
df_london = pd.read_csv(url, encoding='Latin-1')

In [ ]:
df_london

In [ ]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [ ]:
dropdown_year = widgets.Dropdown(options =    unique_sorted_values_plus_ALL(df_london.year))

In [ ]:
def dropdown_year_eventhandler(change):
    if (change.new == ALL):
        display(df_london)
    else:
        display(df_london[df_london.year == change.new])

In [ ]:
dropdown_year.observe(dropdown_year_eventhandler, names='value')


In [ ]:
dropdown_year

In [ ]:
dropdown_year = widgets.Dropdown(options =    unique_sorted_values_plus_ALL(df_london.year))

In [ ]:
output_year = widgets.Output()

In [ ]:
def dropdown_year_eventhandler(change):
    output_year.clear_output()
    with output_year:
        display(df_london[df_london.year == change.new])

In [ ]:
dropdown_year.observe(
dropdown_year_eventhandler, names='value')

In [ ]:
display(dropdown_year)
display(output_year)
